In [15]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('/content/R2.csv')

from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
stop_words = stopwords.words('spanish')

df.columns = ['ID', 'R2', "Conocimiento"]

texts = df['R2'].copy()
texts = [text.lower() for text in texts ]
texts = [text.split() for text in texts ]
texts = [[word.strip() for word in text] for text in texts]
texts = [[word for word in text if word not in stop_words ] for text in texts]

df["words"] = texts





from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

def preprocess_input(df, max_seq_len=None):
  texts = df['R2'].copy()
  labels = df['Conocimiento'].copy()

  stop_words = stopwords.words('spanish')

  texts = [text.lower() for text in texts ]
  texts = [text.split() for text in texts ]
  texts = [[word.strip() for word in text] for text in texts]
  texts = [[word for word in text if word not in stop_words ] for text in texts]

  vocab_length = 10000

  tokenizer = Tokenizer(num_words=vocab_length)
  tokenizer.fit_on_texts(texts)
  texts = tokenizer.texts_to_sequences(texts)

  if max_seq_len is None:
    max_seq_len = np.max([len(text) for text in texts])

  texts = pad_sequences(texts, maxlen=max_seq_len, padding='post')

  labels = np.array(labels)

  return texts, labels, max_seq_len, vocab_length, labels

texts, labels, max_seq_len, vocab_lenght, label_mapping = preprocess_input(df, 22)
print("Frases en el texto:\n", texts.shape)
print("\nEtiquetas:\n", labels.shape)
print("\nLongitud maxima de cada frase:\n", max_seq_len)
print("\nLongitud maxima de vocabulario:\n", vocab_lenght)
print("\nLabel mapping:\n", label_mapping)

Frases en el texto:
 (100, 22)

Etiquetas:
 (100,)

Longitud maxima de cada frase:
 22

Longitud maxima de vocabulario:
 10000

Label mapping:
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
import tensorflow as tf
import numpy as np
from keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout

In [16]:
encoded_labels = np.zeros((len(labels), 3))  # Crear un array de ceros de forma (n, 3)

# Codificar los valores manualmente
for i, label in enumerate(labels):
    encoded_labels[i, label-1] = 1

labels = encoded_labels
texts_train, texts_test, labels_train, labels_test = train_test_split(texts, labels, train_size=0.7, random_state=220)
model = Sequential()
model.add(Embedding(input_dim=vocab_lenght, output_dim= 512, input_length=max_seq_len))
model.add(LSTM(units=128, return_sequences=True))
model.add(Dropout(0.2))  # Regularización mediante Dropout
model.add(LSTM(units=64))
model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=3, activation='softmax'))

# Compilar el modelo
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenar el modelo
model.fit(texts_train, labels_train, epochs=8, batch_size=32)

# Evaluar el modelo en el conjunto de prueba
loss, accuracy = model.evaluate(texts_test, labels_test)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)

Epoch 1/8
3/3 [==============================] - 5s 132ms/step - loss: 1.0975 - accuracy: 0.3714
Epoch 2/8
3/3 [==============================] - 0s 138ms/step - loss: 1.0385 - accuracy: 0.6571
Epoch 3/8
3/3 [==============================] - 0s 120ms/step - loss: 0.9573 - accuracy: 0.6714
Epoch 4/8
3/3 [==============================] - 0s 120ms/step - loss: 0.8314 - accuracy: 0.6714
Epoch 5/8
3/3 [==============================] - 0s 146ms/step - loss: 0.6282 - accuracy: 0.7286
Epoch 6/8
3/3 [==============================] - 1s 203ms/step - loss: 0.3856 - accuracy: 0.7714
Epoch 7/8
3/3 [==============================] - 1s 175ms/step - loss: 0.2752 - accuracy: 0.7714
Epoch 8/8
1/1 [==============================] - 1s 849ms/step - loss: 0.4144 - accuracy: 0.9000
Test Loss: 0.41442352533340454
Test Accuracy: 0.8999999761581421


In [17]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 22, 512)           5120000   
                                                                 
 lstm_4 (LSTM)               (None, 22, 128)           328192    
                                                                 
 dropout_2 (Dropout)         (None, 22, 128)           0         
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 3)                 99        
                                                                 
Total params: 5,499,779
Trainable params: 5,499,779
No